<a href="https://colab.research.google.com/github/Mahe077/Machine-Learning/blob/main/Random_Forest_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import normalize

In [2]:
data = "/content/drive/MyDrive/ML/Random Forest/Assignment_1/pd_speech_features.csv"
df = pd.read_csv(data,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,745,746,747,748,749,750,751,752,753,754
0,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
1,0,1,0.85247,0.71826,0.57227,240,239,0.00806353,8.68E-05,0.00218,...,1.562,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
2,0,1,0.76686,0.69481,0.53966,234,233,0.008258256,7.31E-05,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.178,1
3,0,1,0.85083,0.67604,0.58982,232,231,0.00833959,6.04E-05,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
4,1,0,0.41121,0.79672,0.59257,178,177,0.010857733,0.000182739,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.846,6.265,4.0603,1


In [3]:
# Descriptive statistics for each column
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,745,746,747,748,749,750,751,752,753,754
count,757,757,757,757,757,757,757,757,757,757,...,757,757,757,757,757,757,757,757,757,757
unique,253,3,741,746,749,316,320,756,647,359,...,750,756,753,754,750,753,754,754,755,3
top,125,1,0.82273,0.80616,0.45816,237,236,0.006004477,3.88E-05,0.00076,...,1.5382,4.0251,2.8454,4.2105,4.2973,3.1144,12.595,4.2391,10.0693,1
freq,3,390,3,2,2,9,8,2,3,9,...,2,2,2,2,2,2,2,2,2,564


In [5]:
feature_columns = df.loc[0,0:753]
print(feature_columns)

0                             id
1                         gender
2                            PPE
3                            DFA
4                           RPDE
                 ...            
749    tqwt_kurtosisValue_dec_32
750    tqwt_kurtosisValue_dec_33
751    tqwt_kurtosisValue_dec_34
752    tqwt_kurtosisValue_dec_35
753    tqwt_kurtosisValue_dec_36
Name: 0, Length: 754, dtype: object


In [6]:
data = df.loc[1:]

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,745,746,747,748,749,750,751,752,753,754
1,0,1,0.85247,0.71826,0.57227,240,239,0.00806353,8.68E-05,0.00218,...,1.562,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
2,0,1,0.76686,0.69481,0.53966,234,233,0.008258256,7.31E-05,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.178,1
3,0,1,0.85083,0.67604,0.58982,232,231,0.00833959,6.04E-05,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
4,1,0,0.41121,0.79672,0.59257,178,177,0.010857733,0.000182739,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.846,6.265,4.0603,1
5,1,0,0.3279,0.79782,0.53028,236,235,0.008161574,0.002668863,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x = data.iloc[:,0:754]
x.head()

,0,1,2,3,4,5,6,7,8,9,...,744,745,746,747,748,749,750,751,752,753
1,0,1,0.85247,0.71826,0.57227,240,239,0.00806353,8.68E-05,0.00218,...,1.5466,1.562,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405
2,0,1,0.76686,0.69481,0.53966,234,233,0.008258256,7.31E-05,0.00195,...,1.553,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.178
3,0,1,0.85083,0.67604,0.58982,232,231,0.00833959,6.04E-05,0.00176,...,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666
4,1,0,0.41121,0.79672,0.59257,178,177,0.010857733,0.000182739,0.00419,...,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.846,6.265,4.0603
5,1,0,0.3279,0.79782,0.53028,236,235,0.008161574,0.002668863,0.00535,...,7.8832,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164


In [10]:
y = data.iloc[:,754:]
y.head()

,754
1,1
2,1
3,1
4,1
5,1


In [11]:
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) # 70% training and 30% test

In [12]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

In [13]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

In [14]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train.values.ravel())

RandomForestClassifier()

In [15]:
y_pred=clf.predict(x_test)

In [16]:
from sklearn import metrics

In [17]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8854625550660793


In [18]:
feature_imp = pd.Series(clf.feature_importances_).sort_values(ascending=False)
feature_imp

369    0.016619
126    0.014202
405    0.012758
347    0.010301
585    0.009928
         ...   
554    0.000000
555    0.000000
556    0.000000
557    0.000000
544    0.000000
Length: 754, dtype: float64

In [19]:
feature_columns = np.asarray(feature_columns)

In [20]:
pd.DataFrame(clf.feature_importances_, index=feature_columns).sort_values(by=0, ascending=False)

,0
tqwt_entropy_shannon_dec_12,0.016619
std_delta_delta_log_energy,0.014202
tqwt_entropy_log_dec_12,0.012758
tqwt_energy_dec_26,0.010301
tqwt_stdValue_dec_12,0.009928
...,...
tqwt_meanValue_dec_17,0.000000
tqwt_meanValue_dec_18,0.000000
tqwt_meanValue_dec_19,0.000000
tqwt_meanValue_dec_20,0.000000
